In [ ]:
import pdfplumber as pp
import pandas as pd

In [ ]:
file = pp.open('IOL856475.pdf')
# page = file.pages[13]
table_settings = {
   'vertical_strategy': 'lines',
   'horizontal_strategy': 'lines'
   }

# im = page.to_image()
# im.debug_tablefinder()

In [ ]:
out = []
filter = ['No', 'Planning Order', 'Defect Order', 'Internal Repair Order', 'Defect Order']
for page in file.pages: 
   table = page.extract_table(table_settings)
   if table != None:
      for tab in table:
         if filter == tab:
            pass
         else:
            out.append(tab)
         


   # for table in file.pages[page]:  
   #    out.append(table.extract_table(table_settings))
# print(out)

# out = page.extract_table(table_settings)
label = out[0] # ['No', 'Planning Order', 'Defect Order', 'Internal Repair Order', 'Defect Order']



# converting list to dataframe
# df = pd.DataFrame(out, columns = label)
# df = pd.DataFrame(out)
# df
# export to excel
# df.to_excel('out.xlsx')

In [ ]:
import re
def _filter(data):
   filter1 = re.sub(r"\s+", " ", data)
   filter2 = re.sub(r"(?=[ ]\d{9})", "\n", filter1)
   datax = re.split('\n', filter2)
   if (len(datax) == 1):
      return filter1
   return datax
ramuan = []
a = 0
# for i in out[1:]:
for i in out:
   
   ramuan.append([i[0]])
   ramuan[a].append(_filter(i[1]))
   ramuan[a].append(_filter(i[2]))
   ramuan[a].append(_filter(i[3]))
   ramuan[a].append(_filter(i[4]))
   a += 1

# print(ramuan)
# er = pd.DataFrame(ramuan, columns=filter)
# er
# ex = er.to_excel('anal.xlsx')


In [ ]:
import re
from openpyxl import Workbook
from openpyxl.styles import Alignment
from openpyxl.worksheet.dimensions import ColumnDimension


wb = Workbook()
ws = wb.active

def _merge(list, start, inc):
   baseList = ["A", "B", "C", "D", "E"]

   if len(list) > 0:
      for col in baseList:
         if col not in list:
            # print(col, start, inc)
            print(f"--merge cells [{col}{start}:{col}{start+inc-1}]--")
            ws.merge_cells(f"{col}{start}:{col}{start+inc-1}")

def _insertCell(cell, data):
   ws[cell] = data
   # ws[cell].alignment = Alignment(horizontal='center', vertical='center')


_inc = 0
_rowCurrent = 1
_colEx = []

for row in range(0, len(ramuan)):
   _rowCurrent
   print(f"--[{_rowCurrent}]--")
   for col in range(0, len(ramuan[row])):
      _chr = chr(65 + col)
      if type(ramuan[row][col]) == list:
         _colEx.append(_chr)
         for li in range(0, len(ramuan[row][col])):
            print(f"{_chr}{_rowCurrent+li}", end= ":")
            print(ramuan[row][col][li])
            _insertCell(f"{_chr}{_rowCurrent+li}", ramuan[row][col][li])
         _inc = len(ramuan[row][col])
      else:
         print(f"{_chr}{_rowCurrent}", end= ":")
         print(ramuan[row][col])
         _insertCell(f"{_chr}{_rowCurrent}", ramuan[row][col])
      if col == 4:
         # print(col)
         # print(_inc, _rowCurrent, _colEx)
         _merge(_colEx, _rowCurrent, _inc)
         print('--end col--\n')
         _colEx = []
   if _inc != 0:
      print(f"--next cell is: {_rowCurrent + _inc}")
      _rowCurrent += _inc
      _inc = 0
   else:
      _rowCurrent += 1
   
ColumnDimension(worksheet=ws, bestFit=True)
wb.save('output.xlsx')